In [ ]:
directory = '../input/state-farm-distracted-driver-detection/imgs/train'

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
from keras.applications.imagenet_utils import preprocess_input

import numpy as np

In [ ]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1./255,
    validation_split=0.2
)
image_size = (256, 256)
batch_size = 32
train_generator = datagen.flow_from_directory(
    directory,
    class_mode='categorical',
    target_size=image_size,
    batch_size=batch_size,
    subset='training'
)
val_generator = datagen.flow_from_directory(
    directory,
    class_mode='categorical',
    target_size=image_size,
    batch_size=batch_size,
    subset='validation'
)

In [ ]:
next(val_generator)[0].shape

In [ ]:
base = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base.trainable = False

In [ ]:
model = models.Sequential()
model.add(base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam', # optimizers.RMSprop(lr=2e-5),
    metrics=['acc']
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=560,
    epochs=3,
    validation_data=val_generator,
    validation_steps=140
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], '--')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'], '--')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.save('trans_model_no_fine_tune.h5')

In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam', # optimizers.RMSprop(lr=2e-5),
    metrics=['acc']
)

In [ ]:
model.save('trans_model_fine_tune.h5')

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=560,
    epochs=3,
    validation_data=val_generator,
    validation_steps=140
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], '--')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'], '--')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
img_tensor = next(train_generator)[0][0:1]
print(img_tensor.shape)
plt.imshow(img_tensor[0])
plt.show()

In [ ]:
conv_count = 6
layer_outputs = [layer.output for layer in model.layers[:conv_count]]
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
activation_model.summary()

In [ ]:
activations = activation_model.predict(img_tensor)
first_layer_activation = activations[0]
plt.matshow(first_layer_activation[0, :, :, 3], cmap='viridis')
plt.show()

In [ ]:
layer_names = []
for layer in model.layers[:conv_count]:
    layer_names.append(layer.name)

images_per_row = 16

# Now let's display our feature maps
for layer_name, layer_activation in zip(layer_names, activations):
    # This is the number of features in the feature map
    n_features = layer_activation.shape[-1]

    # The feature map has shape (1, size, size, n_features)
    size = layer_activation.shape[1]

    # We will tile the activation channels in this matrix
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # We'll tile each filter into this big horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
plt.show()